In [ ]:
import sys
sys.path.insert(0, '../')

import uuid
import os
import time
from typing import Any, Callable, Generator

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.base import BaseOutputParser
from langchain.agents import load_tools, Tool
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

from chat.base import BaseChat, ChatHistory, Response
from chat.widget_search import ChatOutputParser, WIDGET_INSTRUCTION, SEARCH_INSTRUCTION, TEMPLATE, IDENTIFY_TEMPLATE
from chat.base import *
from index.weaviate import *
from index.widgets import *
from utils import *
from config import *
from system import *
from tools import *

set_api_key()

def send_message(resp, before_message_id=None, last_chat_message_id=None):
    if resp.operation == 'replace':
        print(resp.response)
    return 1

system = initialize(default_config)

In [ ]:
history = ChatHistory([], uuid.UUID, '0x123243242')
system.chat.receive_input(history, "what is the price of ETH", send_message, uuid.UUID, uuid.UUID)

### Test Models

In [ ]:
TEMPLATE = '''You are a web3 widget tool. You have access to a list of widget magic commands that you can delegate work to, by invoking them and chaining them together, to provide a response to an input query. Magic commands have the structure "<|command(parameter1, parameter2, ...)|>" specifying the command and its input parameters. They can only be used with all parameters having known and assigned values, otherwise, they have to be kept secret. The command may either have a display- or a fetch- prefix. When you return a display- command, the user will see data, an interaction box, or other inline item rendered in its place. When you return a fetch- command, data is fetched over an API and injected in place. Users cannot type or use magic commands, so do not tell them to use them. Fill in the command with parameters as inferred from the input. If there are missing parameters, do not use magic commands but mention what parameters are needed instead. If there is no appropriate widget available, explain that more information is needed. Do not make up a non-existent widget magic command, only use the applicable ones for the situation, and only if all parameters are available. You might need to use the output of widget magic commands as the input to another to get your final answer. Here are the widgets that may be relevant:
---
Widget magic command: <|fetch-price({baseToken},{quoteToken})|>
Description of widget: This widget is used to get the price of a token. Note, when the quoteToken isn't explicitly specified assume it to be USD.
Required parameters:
-{baseToken}: token to get the price of
-{quoteToken}: token to use as units for price
Return value description:
-price of a base token in units of a quote token.
---
Widget magic command: <|fetch-my-balance({token})|>
Description of widget: This widget is used when we need the balance of a token in the user's connected wallet
Required parameters:
-{token}: token to get the balance of.
Return value description:
-balance of a token in connected wallet, in decimal units
---
Use the following format:

## Tool Input: given a query which you have to rephrase, explicitly restating the task without pronouns and restating details based on the conversation history and new input. Restate verbatim ALL details/names/figures/facts/etc from past observations relevant to the task and ALL related entities.
## Widget Command: most relevant widget magic command to respond to Tool Input
## Known Parameters: parameter-value pairs representing inputs to the above widget magic command
## Response: return the widget magic command with ALL its respective input parameter values (omit parameter names)

## Tool Input: I want to get the price of Ethereum in terms of USD.
## Widget Command:'''

In [ ]:
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks import streaming_stdout
from text_generation import Client

callbacks = [streaming_stdout.StreamingStdOutCallbackHandler()]
streaming_kwargs = dict(
            stream=True,
            callbacks=callbacks,
        ) 

inference_server_url = HUGGINGFACE_INFERENCE_ENDPOINT
headers = {
    "Authorization": f"Bearer {HUGGINGFACE_API_KEY}",
    "Content-Type": "application/json",
}
client = Client(inference_server_url, headers=headers)
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=10,
    temperature=0.1, # should be strictly positive
    **streaming_kwargs,
)
llm.client = client
print(llm("What is Deep Learning?"))